In [1]:
import numpy as np
import nibabel as nib
import os

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize
from pylab import rcParams

from sklearn.preprocessing import scale, StandardScaler
from sklearn.linear_model import Lasso, Ridge

import pandas as pd

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error as MAE

import gc

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.models import Model
from keras import backend as K

def make_age_list(files, age_df):
    age_list = []
    for file in files:
        ID = file[17:25]
        age_list.append(df.loc[df[0] == ID][1].values[0])
    return np.array(age_list)

Using TensorFlow backend.


In [2]:
def preprocess_data(data, k, axis = 1):
    #k - index of slice we are planning to use
    #axis - type of slice (axial, sagittal, coronal)
    
    
    #This is to normalize data before inputing it to NN
    std = StandardScaler()
    
    # we make a mask to zero everythin with variance below some reasonable point
    if axis == 0:
        mask = np.var(data[:,k,:,:],0)
    elif axis == 1:
        mask = np.var(data[:,:,k,:],0)
    else:
        mask = np.var(data[:,:,:,k],0)
    mask[mask < 0.000001] = 0
    mask = ~mask.astype(bool)
    
    
    if axis == 0:
        new_data = np.zeros((data.shape[0], 145, 121, 3))
        for i in range(3):
            for j in range(121):
                new_data[:,j,:,i] = std.fit_transform(data[:,k-10+10*i,j,:])
            new_data[:,mask,i] = 0
        new_data = new_data[:,12:133,:,:]
    elif axis == 1:
        new_data = np.zeros((data.shape[0], 121, 121, 3))
        for i in range(3):
            for j in range(121):
                new_data[:,j,:,i] = std.fit_transform(data[:,j,k-10+10*i,:])
            new_data[:,mask,i] = 0
    elif axis == 2:
        new_data = np.zeros((data.shape[0], 121, 145, 3))
        for i in range(3):
            for j in range(121):
                new_data[:,j,:,i] = std.fit_transform(data[:,j,:,k-10+10*i])
            new_data[:,mask,i] = 0
        new_data = new_data[:,:,12:133,:]
    return new_data.astype(np.float16)

In [3]:
#Reading the files
folder_wm = 'wm_data_big'
folder_gm = 'gm_data_big'

df = pd.read_csv('BANC_2016.csv', header=None, sep=',')

files_wm = os.listdir(folder_wm)
files_gm = os.listdir(folder_gm)
files_wm = sorted(files_wm)
files_gm = sorted(files_gm)

age = df[3]

data = np.zeros((len(files_wm), 121,145,121), dtype = np.float32)

for sub in tqdm(range(len(files_wm))):
    ID = files_wm[sub]
    img = nib.load(folder_wm + '/'+ID)
    data[sub,...] = img.get_data()
    
    
#prep_data is a list of preprocessed data. It will have length of 6 - 
#thirst 3 elements are 3 middle slices(axial, coronal, sagittal) of WM, next the same of GM

prep_data = []
prep_data.append(preprocess_data(data,60, 0))
prep_data.append(preprocess_data(data,72, 1))
prep_data.append(preprocess_data(data,60, 2))
data = []

#This is garbage collector, it clears RAM. Without this the code crashes on 32 GB of RAM, so we need this every
#time we clear some variable, like just before a moment. 
gc.collect()

data = np.zeros((len(files_wm), 121,145,121), dtype = np.float32)

for sub in tqdm(range(len(files_gm))):
    ID = files_gm[sub]
    img = nib.load(folder_gm + '/'+ID)
    data[sub,...] = img.get_data()
    
prep_data.append(preprocess_data(data,60, 0))
prep_data.append(preprocess_data(data,72, 1))
prep_data.append(preprocess_data(data,60, 2))
data = []
gc.collect()

0

In [9]:
# Baseline Ridge - we just use Ridge regression without pushig data trrough Xception. The result is not that bad


kf = KFold(10)
predicted_ages = np.zeros(age.shape[0])
model = Ridge()
features = np.concatenate([prep_data[0], prep_data[1], 
                              prep_data[2], prep_data[3],
                          prep_data[4], prep_data[5]], 1)
features = features.reshape(features.shape[0], np.prod(features.shape[1:]))
features = features[:,:]
for i_train, i_test in tqdm(kf.split(features, age)):
    model.fit(features[i_train,:], age[i_train])
    predicted_ages[i_test] = model.predict(features[i_test,:])
print(MAE(age, predicted_ages))


5.04090604312


In [4]:
#We make fatures by pushing our slices through 19 layers of xception

xception = Xception(include_top=False, weights='imagenet', 
                        input_tensor=None, input_shape=(121,121,3), pooling=None, classes=1000)


xception_model = Model(inputs=xception.input, outputs=xception.get_layer('block14_sepconv2_act').output)
inp = xception_model.input                                           # input placeholder
outputs = [xception.layers[19].output]     # all layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function

xception_features = []

f_batch = 256
features = np.zeros((2001, 14, 14, 256, 6))

for i in tqdm(range(6)):
    temp = np.zeros([prep_data[0].shape[0],] + xception.layers[19].output.get_shape().as_list()[1:])
    for b in range((prep_data[0].shape[0]-1)//f_batch+1):
        temp[f_batch*b:f_batch*(b+1), ...] = functor(
            [prep_data[i][f_batch*b:f_batch*(b+1),:121,:121,:], 1.])[0]

    #Here is max pooling. This should definitely be done in keras, but I was under heavy time constraint and this
    #was just faster to code
    temp = temp.reshape(temp.shape + (1,))
    temp = np.max(np.concatenate([temp[:,2::2,...], temp[:,1::2,...]] , 4), 4)
    temp = temp.reshape(temp.shape + (1,))
    temp = np.max(np.concatenate([temp[:,:,:28:2,...], temp[:,:,1::2,...]] , 4), 4)
    features[...,i] = temp
temp = []
gc.collect()

49140

In [5]:
# Ridge on "Xception preprocessed" data
gc.collect()

kf = KFold(20, shuffle = True)

predicted_ages = np.zeros(2001)

model = Ridge()
features = features.reshape(features.shape[0], np.prod(features.shape[1:]))
# features = features[:,::3]
gc.collect()
for i_train, i_test in tqdm(kf.split(features, age)):
    model.fit(features[i_train,:], age[i_train])
    predicted_ages[i_test] = model.predict(features[i_test,:])
print(MAE(age, predicted_ages))


4.17975930152
